# State Construction Task

In [173]:
from datetime import datetime
import itertools
import io

import numpy as np
import scipy.sparse as sp
import tensorflow as tf

from tqdm import tqdm
import matplotlib.pyplot as plt

simple utilities for complex mapping

In [174]:
def encode(t):
    if t.ndim == 1:
        t = np.expand_dims(t, axis=-1)
    return np.vstack([np.hstack([t.real, -t.imag]),
        np.hstack([t.imag, t.real])])

def decode_shape(shape):
    assert(shape[0] % 2 == 0 and shape[1] % 2 == 0)
    return (shape[0] // 2, shape[1] // 2) + shape[2:]

def decode(t):
    rshape = decode_shape(t.shape)
    r = t[0:rshape[0], 0:rshape[1]] + 1j * t[rshape[0]:2 * rshape[0], 0:rshape[1]]
    if r.ndim == 2 and rshape[1] == 1:
        r = r[:, 0]
    return r


generate a parametric hamiltonian with `tf.Variable`

In [175]:
def generate_hamiltonain(dim):
    def generate_parametric(n, dv_lambda):
        params = tf.Variable(tf.random_normal((n, )))
        places = tf.constant(np.stack([dv_lambda(c) for c in range(n)]), dtype='float32')
        return tf.tensordot(places, params, axes=[0, 0])
    

    neardiag = lambda i : encode(sp.coo_matrix(
        ([1, 1], ([i, (i + 1) % dim], [(i + 1) % dim, i % dim])), shape=(dim, dim)).toarray())

    diag = lambda i : encode(sp.coo_matrix(([1], ([i], [i])), shape=(dim, dim)).toarray())
    h = generate_parametric(dim - 1, neardiag) + generate_parametric(dim, diag)
    
    return h

implement the optimization problem as a computation graph

In [176]:
def generate_graph(dim):
    source = tf.constant(encode(np.array([1] + [0] * (dim - 1))), dtype='float32')
    target_vec = np.random.normal(size=(dim,)) 
    target_vec /= np.linalg.norm(target_vec)
    target= tf.constant(encode(target_vec), dtype='float32')

    i_mat = tf.constant(encode(1j * np.eye(dim)), dtype='float32')
    h = generate_hamiltonain(dim)
    gate = tf.linalg.expm(tf.matmul(i_mat, h))
    estimated_target = tf.matmul(gate, source)

    target_dot = tf.matmul(tf.transpose(target), estimated_target)
    fidelity = tf.sqrt(2 * tf.reduce_mean(tf.matmul(tf.transpose(target_dot), target_dot)))
    loss = 1 - fidelity

    summary_op = tf.summary.scalar('fidelity', fidelity)
    optimize_op = tf.train.MomentumOptimizer(momentum=0.8,learning_rate=0.5).minimize(loss)
    return optimize_op, summary_op

storing metrics (using tensorboard)

In [170]:
def plot_matrix(h):
    figure = plt.figure(figsize=(8, 8))
    plt.imshow(h, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title("Matrix")
    plt.colorbar()

    threshold = h.max() / 2.
    for i, j in itertools.product(range(h.shape[0]), range(h.shape[1])):
        color = "white" if h[i, j] > threshold else "black"
        plt.text(j, i, h[i, j], horizontalalignment="center", color=color)

    plt.tight_layout()
    return figure

def plot_to_image(figure):
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    plt.close(figure)
    buf.seek(0)
    image = tf.image.decode_png(buf.getvalue(), channels=4)
    image = tf.expand_dims(image, 0)
    return image

run the whole

In [183]:
folder = './tensorboard_logs/%s' % datetime.now().isoformat()

tf.reset_default_graph()
with tf.Session() as session:
    
    for dim in [4, 8, 16, 32]:
        optimize_op, summary_op = generate_graph(dim)
        writer = tf.summary.FileWriter(folder + '/%d' % dim, session.graph)
        init_op = tf.global_variables_initializer()
        session.run(init_op)
        for i in tqdm(range(5000)):
            _, summary_val = session.run([optimize_op, summary_op])
            writer.add_summary(summary_val, i)

100%|██████████| 5000/5000 [00:04<00:00, 1008.71it/s]
